<a href="https://colab.research.google.com/github/aopensensei/sample/blob/main/XLSR_JP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

参考

Fine-Tune XLSR-Wav2Vec2 on Turkish ASR with 🤗 Transformers.ipynb
　https://colab.research.google.com/github/patrickvonplaten/notebooks/blob/master/Fine_Tune_XLSR_Wav2Vec2_on_Turkish_ASR_with_%F0%9F%A4%97_Transformers.ipynb

wav2vec 2.0を日本語で推論できるようにする
　https://tech.fusic.co.jp/posts/2021-03-30-wav2vec-fune-tune/


In [ ]:
%%capture
!pip install datasets==1.13.3
!pip install transformers==4.11.3
!pip install torchaudio
!pip install librosa
!pip install jiwer

In [ ]:
#何かと使うので忘れないうちに
import numpy as np
import pandas as pd

In [ ]:
#Common Voice の 日本語コーパスから
from datasets import load_dataset, load_metric, Audio

common_voice_train = load_dataset("common_voice", "ja", split="train+validation")
common_voice_test = load_dataset("common_voice", "ja", split="test")

Downloading:   0%|          | 0.00/4.44k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/153M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset common_voice downloaded and prepared to /root/.cache/huggingface/datasets/common_voice/ja/6.1.0/f7a9d973839b7706e9e281c19b7e512f31badf3c0fdbd21c671f3c4bf9acf3b9. Subsequent calls will reuse this data.


Reusing dataset common_voice (/root/.cache/huggingface/datasets/common_voice/ja/6.1.0/f7a9d973839b7706e9e281c19b7e512f31badf3c0fdbd21c671f3c4bf9acf3b9)


In [ ]:
# 中身確認
print(common_voice_train)
print(common_voice_test)

print(common_voice_train['sentence'][:10])

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 1308
})
Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 632
})
['予想外の事態に、電力会社も、ちょっぴり困惑気味だ。', '町域にあった三根山藩は長岡藩に米百俵を送ったことで有名', '週末、友達と山に登ります。', 'あとで図書館へ本を返しに行きます。', '五十五歳だって、うれしいときはうれしいのだ。', 'わたしはパンもごはんも好きです。', 'デパートやスーパーで買い物をします。', '用紙に書いてある番号を覚えます。', 'あした友達と映画を見に行きます。', 'あの男の人は背が高くて、足が長いです。']


In [ ]:
# データセットから必要なカラムのみを残して削除
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [ ]:
#ランダム中身確認用関数
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
#中身確認
show_random_elements(common_voice_train.remove_columns(["path", "audio"]), num_examples=10)

,sentence
0,わたしはミルクを入れないで、コーヒーを飲みます。
1,今、午後七時です。
2,ドラマは、あくまでもドラマであり、虚構であって、現実ではない。
3,イさんにこのことばの意味を教えました。
4,わたしは妹の宿題を見てあげました。
5,机の上に紙と鉛筆があります。
6,今晩は長い時間勉強しようと思っています。
7,人間の心底に潜む、自分本位の醜さを、遠慮なくさらけ出します。
8,熱があるので、きょうは学校を休みます。
9,イさんは日本語で手紙を書きました。


In [ ]:
#不要文字削除関数
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'
  
def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [ ]:
#不要文字削除
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

  0%|          | 0/1308 [00:00<?, ?ex/s]

  0%|          | 0/632 [00:00<?, ?ex/s]

In [ ]:
#中身確認
show_random_elements(common_voice_train.remove_columns(["path","audio"]))

,sentence
0,駅を出て、あの道をまっすぐ行ってください。
1,わたしの弟は九歳です。
2,夏になると、暑くなります。
3,松井さんは静かな町に住んでいます。
4,山の裾に休息している、憐れな漁村の屋根を見た。
5,プールの水は冷たいです。
6,携帯電話は小さくて、とても便利です。
7,友達の自転車のうしろに乗りました。
8,きのうペットと遊びました。
9,家へ帰ったら、初めに宿題をします。


In [ ]:
#テキストから文字リストを作成する関数
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
#文字リスト作成
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
#重複排除して結合
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [ ]:
#文字辞書作成
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'喜': 0,
 '痴': 1,
 'あ': 2,
 '洗': 3,
 '招': 4,
 '掃': 5,
 '郊': 6,
 '諸': 7,
 'レ': 8,
 '産': 9,
 '異': 10,
 '減': 11,
 '膝': 12,
 'ツ': 13,
 '筆': 14,
 '賃': 15,
 '会': 16,
 'フ': 17,
 '明': 18,
 '写': 19,
 '伸': 20,
 'チ': 21,
 '失': 22,
 'で': 23,
 '士': 24,
 '装': 25,
 '阪': 26,
 '馬': 27,
 '都': 28,
 '希': 29,
 '、': 30,
 '界': 31,
 '演': 32,
 '印': 33,
 '懸': 34,
 '軽': 35,
 '宝': 36,
 '片': 37,
 '〇': 38,
 '検': 39,
 '娘': 40,
 '紅': 41,
 '振': 42,
 '舞': 43,
 '仕': 44,
 '露': 45,
 '栓': 46,
 '厚': 47,
 '宮': 48,
 '岡': 49,
 'へ': 50,
 '型': 51,
 '誕': 52,
 '犠': 53,
 '初': 54,
 '短': 55,
 '所': 56,
 '賢': 57,
 '叩': 58,
 'ナ': 59,
 '冬': 60,
 '期': 61,
 'デ': 62,
 '標': 63,
 'す': 64,
 'ル': 65,
 'ピ': 66,
 '打': 67,
 '却': 68,
 '散': 69,
 '徹': 70,
 '少': 71,
 '差': 72,
 'げ': 73,
 '強': 74,
 '衣': 75,
 '果': 76,
 '安': 77,
 '言': 78,
 '借': 79,
 'き': 80,
 '観': 81,
 '悲': 82,
 '夫': 83,
 '富': 84,
 '機': 85,
 '物': 86,
 'テ': 87,
 '松': 88,
 '備': 89,
 '逆': 90,
 'お': 91,
 '店': 92,
 '何': 93,
 '広': 94,
 '眠': 95,
 '械': 96,
 '真': 97,
 'マ': 98,
 '存': 99,
 '財': 100,

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

1436

In [ ]:
#辞書保存
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
#トークナイザー
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
#特徴量抽出器
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
#プロセッサー(Wav2Vec2の特徴抽出機とトークナイザーをまとめてラップ)
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
#google drive上に保存したいとき
#from google.colab import drive
#drive.mount('/content/gdrive/')

In [ ]:
#元の音声ファイルへのパス確認用
common_voice_train[0]["path"]

'/root/.cache/huggingface/datasets/downloads/extracted/07b0a73f2103df267f566548f7597fe8d75f8d4bdd37b7f556478ae85378bd6a/cv-corpus-6.1-2020-12-11/ja/clips/common_voice_ja_19817895.mp3'

In [ ]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

In [ ]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["sentence"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

Target text: 美容院は駅の横にあります。 
Input array shape: (69504,)
Sampling rate: 16000


In [ ]:
# 
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [ ]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names, num_proc=4)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names, num_proc=4)

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")

Downloading:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Downloading:   0%|          | 0.00/1.73k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['project_hid.bias', 'quantizer.weight_proj.weight', 'project_q.weight', 'quantizer.codevectors', 'quantizer.weight_proj.bias', 'project_hid.weight', 'project_q.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to u

In [ ]:
model.freeze_feature_extractor()

In [ ]:
model.gradient_checkpointing_enable()

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  #output_dir="/content/gdrive/MyDrive/wav2vec2-large-xlsr-turkish-demo",
  output_dir="./wav2vec2-large-xlsr-japanese-demo",
  group_by_length=True,
  #メモリが足りなくなるのでパッチサイズを16→8
  per_device_train_batch_size=8,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

Using amp fp16 backend


In [ ]:
#時間がかかると切断されるので、ChromeでF12押して、コンソールに下記を張り付けて、定期的にアクセスさせる
"""

function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);

"""

'\n\nfunction ConnectButton(){\n    console.log("Connect pushed"); \n    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() \n}\nsetInterval(ConnectButton,60000);\n\n'

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 1308
  Num Epochs = 30
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 2460
/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:882: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, round

Step,Training Loss,Validation Loss,Wer
100,44.153600,39.672779,1.000000
200,7.760500,7.113694,1.000000
300,5.125500,5.324620,1.000000
400,4.973300,4.943129,1.000000
500,4.938000,4.860403,1.990506
600,4.692800,4.759236,2.371835
700,4.661200,4.694860,2.238924
800,4.266100,4.683003,2.681962
900,3.983600,4.612966,2.969937


***** Running Evaluation *****
  Num examples = 632
  Batch size = 8
Saving model checkpoint to ./wav2vec2-large-xlsr-japanese-demo/checkpoint-100
Configuration saved in ./wav2vec2-large-xlsr-japanese-demo/checkpoint-100/config.json
Model weights saved in ./wav2vec2-large-xlsr-japanese-demo/checkpoint-100/pytorch_model.bin
Configuration saved in ./wav2vec2-large-xlsr-japanese-demo/checkpoint-100/preprocessor_config.json
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:882: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1
***** Running Evaluation *****
  Num examples = 632
  

In [ ]:
#推論する音声ファイルパス
path = "/root/.cache/huggingface/datasets/downloads/extracted/07b0a73f2103df267f566548f7597fe8d75f8d4bdd37b7f556478ae85378bd6a/cv-corpus-6.1-2020-12-11/ja/clips/common_voice_ja_19817895.mp3"


In [ ]:
#16000に
import torchaudio
import librosa

def load_and_resample(file_path, resamling_rate=16000):
    signal_tensor, samling_rate = torchaudio.load(file_path)
    signal_array = signal_tensor[0].numpy()
    signal_array = librosa.resample(signal_array, 48000, resamling_rate)
    return signal_array

In [ ]:
#推論
signal = load_and_resample(path)
signal_list = signal.tolist()

input_dict = processor(signal_list, return_tensors="pt", padding=True, sampling_rate=16000)
logits = model(input_dict.input_values.to("cuda")).logits
pred_ids = torch.argmax(logits, dim=-1)[0]

transcription = processor.decode(pred_ids)
transcription